In [6]:
import datetime
import glob
import os

from herbie import Herbie
import numpy as np
import pandas as pd
from pvlib import location, solarposition, irradiance

import sys
sys.path.insert(0, os.path.join("..", "AMY"))
import constants, utils

latitude = 29.25,  # N
longitude = 360-98.31,  # W


In [2]:
H = Herbie("2022-10-11 00:00")

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2022-Oct-11 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


In [7]:
ds = H.xarray("(:UGRD:10 m above ground:anl)|(:VGRD:10 m above ground:anl)")
ds

/Users/jeh/anaconda3/envs/amy/lib/python3.8/site-packages/herbie/core.py:797: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  logic = df.search_this.str.contains(searchString)


<xarray.Dataset>
Dimensions:              (y: 1059, x: 1799)
Coordinates:
    time                 datetime64[ns] 2022-10-11
    step                 timedelta64[ns] 00:00:00
    heightAboveGround    float64 10.0
    latitude             (y, x) float64 21.14 21.15 21.15 ... 47.86 47.85 47.84
    longitude            (y, x) float64 237.3 237.3 237.3 ... 299.0 299.0 299.1
    valid_time           datetime64[ns] 2022-10-11
Dimensions without coordinates: y, x
Data variables:
    u10                  (y, x) float32 -6.89 -6.89 -6.828 ... 6.36 6.36 6.36
    v10                  (y, x) float32 -3.176 -3.301 -3.363 ... 5.137 5.262
    gribfile_projection  object None
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    model:                   hrrr
    product:                 sfc
    description:             High-Resolution Rapid Refresh - CONUS
    remote_grib:             https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr....
    local_grib:              /Users/jeh/data/hrrr/20221011/subset_94ef391b__h...
    searchString:            (:UGRD:10 m above ground:anl)|(:VGRD:10 m above ...

In [19]:
latitudes = ds.latitude.values
longitudes = ds.longitude.values
distance = (ds.longitude.values - longitude)**2 + (ds.latitude.values - latitude)**2
argmin_y, argmin_x = divmod(distance.argmin(), distance.shape[1])

In [29]:
x_lon = (longitudes[argmin_y, argmin_x+1] - longitudes[argmin_y, argmin_x-1]) / 2
x_lat = (latitudes[argmin_y, argmin_x+1] - latitudes[argmin_y, argmin_x-1]) / 2
y_lon = (longitudes[argmin_y+1, argmin_x] - longitudes[argmin_y-1, argmin_x]) / 2
y_lat = (latitudes[argmin_y+1, argmin_x] - latitudes[argmin_y-1, argmin_x]) / 2

u_lon = x_lon / (x_lon**2 + x_lat**2)**0.5
v_lon = y_lon / (y_lon**2 + y_lat**2)**0.5
u_lat = x_lat / (x_lon**2 + x_lat**2)**0.5
v_lat = y_lat / (y_lon**2 + y_lat**2)**0.5

In [27]:
u_lon, v_lon

(0.9999712998197906, 0.009952709059404449)

In [28]:
latitudes[:,0]

array([21.138123  , 21.16299459, 21.18786863, ..., 47.78955926,
       47.81409316, 47.8386235 ])